# Análise de resumos por grandes áreas do conhecimento

In [ ]:
from pathlib import Path
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn.objects as so
from seaborn import axes_style

tqdm.pandas()

workdir = Path("..")
df = pd.read_parquet(
    workdir / "data" / "catalogo-de-teses-e-dissertacoes-limpo.parquet",
    columns=[
        "NM_PRODUCAO",
        "DS_RESUMO",
        "DS_RESUMO_LIMPO",
        "CD_GRANDE_AREA_CONHECIMENTO",
        "NM_GRANDE_AREA_CONHECIMENTO",
    ],
)
df.shape

Antes de começar a análise, vamos fazer uma rápida exploração dos dados. 

In [ ]:
# Checa se há divergência entre o nome e o código da grande área de conhecimento
df.groupby("NM_GRANDE_AREA_CONHECIMENTO")["CD_GRANDE_AREA_CONHECIMENTO"].nunique()

Não há dados nulos nas coluna selecionadas.

In [ ]:
df.info()

Por fim, exibimos no gráfico abaixo a distribuição de resumos por área do conhecimento. Notamos que a área de Linguística, Letras e Artes é a que tem menos resumos, enquanto a área de Ciências Humanas é a que tem mais resumos.

In [ ]:
fig = plt.figure(figsize=(8, 4))

thesis_count = df["NM_GRANDE_AREA_CONHECIMENTO"].value_counts().to_frame()

(
    so.Plot(
        thesis_count,
        x="count",
        y="NM_GRANDE_AREA_CONHECIMENTO",
        color=["top"] * 3 + ["demais"] * (len(thesis_count) - 3),
        text=thesis_count["count"]
        .apply(lambda x: f"{x:,}")
        .str.replace(",", ".")
        .values,
    )
    .add(so.Bar(color="#F1F1F1"), legend=False)
    .add(so.Text(color="k", halign="right", offset=-45))
    .label(
        x="",
        y="",
    )
    .limit(x=(0, thesis_count["count"].max() + 0.2 * thesis_count["count"].max()))
    .theme(
        {
            **axes_style("whitegrid"),
            "grid.linestyle": ":",
            "axes.spines.top": False,
            "axes.spines.right": False,
            "font.family": "sans-serif",
            "font.size": 10,
        }
    )
    .on(fig)
    .plot()
)

ax = fig.axes[0]

# colore as três primeiras barras
for i in range(3):
    ax.patches[i].set_color("#FFCC02")

# Título e fonte
fig.text(
    0.12,
    1.09,
    "Distribuição por grande área de conhecimento",
    va="center",
    fontsize=14,
    fontweight="bold",
    fontname="sans-serif",
    color="#333",
)

fig.text(
    0.12,
    1,
    "Ciências humanas, ciências da saúde e ciências sociais aplicadas\nsão as áreas com mais teses e dissertações defendidas.",
    va="center",
    fontsize=12,
    fontname="sans-serif",
    color="#666",
)

fig.text(
    0.75,
    -0.01,
    "Fonte: Catálogo de teses e dissertações da CAPES",
    ha="center",
    va="center",
    fontsize=10,
    fontname="sans-serif",
    color="#999",
)

fig.show()